# Measurement Overview for SAXSess

Takes multiple groups of measurement IDs and generates a combined plot with the group name as title for each group of IDs.

*Author: Ingo Breßler (ingo.bressler@bam.de)*  
*Last modified: 2025-07-28*

## Module imports and preparation

In [ ]:
from pathlib import Path
from pprint import pprint
import datetime, re
from pathvalidate import sanitize_filename
from jupyter_analysis_tools import readPDH, readSSFZ, plotPDH, createFigure, naturalKey

## Provide path and measurement IDs

In [ ]:
measurements = dict()
measurements["Plot Title 1"] = (
    "S00212", "S00188", "S00190", "S00192", "S00195", "S00196", "S00197", "S00250", "S00206"
)
measurements["Plot Title 2"] = (
    "S00212", "S00221", "S00222", "S00223", "S00225", "S00226", "S00227", "S00228", "S00229",
    "S00230", "S00231", "S00232", "S00233", "S00234", "S00235", "S00236", "S00237", "S00238",
    "S00239", "S00240", "S00241", "S00242", "S00243", "S00244", "S00245", "S00246", "S00247",
    "S00248", "S00249", "S00250"
)
datapath = r"R:\FB65\PAZ\2024\12345"
outPath = Path.home() / "Downloads"  # output path for plot images

## Check given IDs

In [ ]:
for plot, measids in measurements.items():
    for measid in measids:
        try:
            if not isinstance(measid, str):
                raise ValueError(f"Expected a string for '{measid}'! Quotes missing?")
            if int(measid[1:]) <= 0:
                raise ValueError(f"Given '{measid}' does not contain a valid positive measurement ID!")
        except ValueError as e:
            print(f"Given ID {measid} in {plot} does not match the expected format, please fix and retry!")
            raise e

## Find the files by title and measurement ID

In [ ]:
datapath = Path(datapath)
searchPaths = [datapath]
i = 0
measPaths = dict()
sessions = dict()
while i < len(searchPaths):
    searchPath = searchPaths.pop(0)
    if searchPath.name[:1] in (".", "_"):
        continue
    #print(f"looking in {searchPath}")
    # collect directories to look at
    for path in searchPath.iterdir():
        if path.is_dir():
            searchPaths.append(path)
            continue
        # see if the file matches one of the measurement numbers/IDs
        thisid = None
        for plot, measids in measurements.items():
            if plot not in measPaths:
                measPaths[plot] = {}
            filepath = {}
            for measid in measids:
                if measid not in path.name:
                    continue
                if measid not in measPaths[plot]:
                    measPaths[plot][measid] = []
                measPaths[plot][measid].append(path)
                thisid = measid
                #print(f"{measPaths[plot][measid]=}")
        if thisid is None:
            continue  # current file does not match one of the measurement IDs
        #print(f"{thisid=}", path)
        # update sessions
        # for pathSSFZ in path.parent.glob("*.[sS][sS][fF][zZ]"):
        #    if pathSSFZ.name in sessions:
        #        continue
        #    sessions[pathSSFZ.name] = readSSFZ(pathSSFZ)
        # operations of this file
        #print(list(sorted(data["session"]["process"]["samples"]["operation"].items(), key=lambda item: item[1]["index"])
        #           for filename, data in sessions.items()
        #           if f">>{thisid}" in data["session"]["process"]["samples"]["declaration"]["reference"]))

print("Found the following files:")
pprint(measPaths)

## Plot file groups

In [ ]:
for title, measids in measPaths.items():
    fig = createFigure(quiet=True)
    ax = fig.add_subplot(111)
    ax.set_title(title)
    print(f"Plotting '{title}':")
    for measid, filepaths in measids.items():
        # take the last, should be the desmeared
        filepath = sorted(filepaths, key=lambda fp: naturalKey(fp.name))[-1]
        label = filepath.stem.split("[")[0]  # strip any '[99]' suffix
        plotPDH(filepath, label, ax=ax, print_filename=False)
        print(f"    File: {filepath.relative_to(datapath)}")
    ax.get_legend().remove()
    fig.legend(loc='outside right center')
    if not outPath.is_dir():
        print(f"Can't store plot in '{outPath}', does not exist!")
        continue
    plotPath = outPath / sanitize_filename(datetime.datetime.now().strftime(f"{title} %y%m%d-%H%M%S.png"))
    fig.savefig(plotPath)
    print(f"-> Exporting plot to '{plotPath}'")